# 1. Topic

<br>
For more than 30 years, the need to reduce energy consumption and greenhouse gas emitions have become more and more of a concern in every business sector, but especially in the transport one.

In that sense, the ADEME (French Environment and Energy Management Agency) has recently launched a call for expression of interests to promote the execution of demos and experiments of new mobility solutions for people and goods, adapted to different kind of territories.

But in fact, those new mobility solutions and new technologies linked to the transport sector in general, although cost-effective and cleaner than older ones, pose new challenges particularly in terms of resource management optimization.

As part of the team set up by CesiCDP, a structure already established in the field of Smart Multimodal Mobility, you know that the current objective is to win new markets with very attractive financing schemes in order to keep developing the team’s business activity. Therefore, CesiCDP took the decision to respond to ADEME’s call for expression of interest and focus its study on the management of delivery route. The structure will indeed try to find an algorithm that could calculate a route on a road network that allows to interconnect a subset of cities to each other and then to return to its starting point. All of this in a minimum of time and taking into account the expected traffic on each axis for the different time slots. The idea the to propose a method from Operations Research to generate a delivery route correponding to this problem.

A basic version of the problem have been outlined yet but you have been thinking that a more complex version of it could make it more realistic and get ADEME to look a bit more into it. You will then have to figure out some additional constraints to implement to the problem.

# 2. Modeling

<br>
We want to minimize the total time spent while visiting all customers. We can now predict that our objective function to minimize will be computing total time spent. At the same time, we want to minimize the number of trucks sent.

<br>

## 2.1 - Function to maximize/minimize

---

→ Minimize the time spent

→ Minimize the number of truck used

→ Minimize the number of truck refill (optional)

## 2.2 - Parameters

---

**Graph**

$G = (C,E)$

- $C = \{ 0,1,2,...,n \}$ : set of the $n+1$  customers with $0$ as the hangar.
- $E$  : edges (routes) connecting the customers.

**Demand**

$w_c$ : Demand (in weight unit) (of customer $c$)

**Time Window**

$(dt^0_c, dt^1_c)$ : Delivery time window (of customer $c$)

$d_c$ : Delivery duration time (of customer $c$)

**Trucks**

$T=\{0,1,...,m\}$ : set of $m$ trucks

$w_t$ : Max weight capacity (of truck $t$)

**Starting Time**

$s_t$ : Starting time of truck $t$

**Distance**

$\forall(i,j) \in C^2 \quad t_{ij}$ : Time taken to go from customer $i$ and customer $j$

**Decision Variable**

$\forall i,j \in C$

$x_{ij}^t\in \{0,1\}$ : Edge need to be taken or not (by truck $t$) between customer $i$ and customer $j$

$a_{i}^t$ : Arriving time of truck $t$ at the customer $c$ ’s place. **$a_i^t$ is a decision variable because we have to decide whenever $a_0^t$ we start our truck and so at which time we want to arrive.**

$\forall i, j\in C^2  \quad i \neq j \quad \forall t \in T \quad \quad a_{j}^t \geq a_{i}^t + d_i + t_{ij}$ : If a truck $t$ travels along the arc $(i,j)$, then its arrival time at $j$ is at least equal to the arrival time at $i$, plus the delivery time $d_i$, and the travel time $t_{ij}$.

$a_{0}^t$ : Departure time of the truck $t$ at the hangar.
<br>

## 2.3 - Constraints (default constraints of VRP - additional constraints)

---

**All client must be visited once**

For that, all node must have a arriving and a departure.

$$
\forall i \in V, i \not\in \{0,j\} \quad \quad \sum_{t\in T} \sum_{j \in V} x_{ij}^t = 1
$$

**Flux Conservation**

Here, we specify that the node must have as many arrives as departures, which must be equal to 1 (with the previous constraint).

$$
\forall i\in C \quad \forall t \in T \quad \quad \sum_{j \in C} x_{ji}^t = \sum_{j' \in C} x_{ij'}^t
$$

**Leaving / Getting back to hangar**

$$
\forall t\quad \sum_{j \in V} x_{0j}^t \geq 1
$$

$$
\forall t\quad \sum_{i \in V} x_{i0}^t \geq 1
$$

**Truck capacity**

$$
\forall t \quad \quad
(\sum_{i \in C}  \sum_{j \in C} w_i * x_{ij}^t) \leq w_t
$$

**Time Window**

$$
\forall i, j\in C^2 \quad i\neq j \quad \forall t \in T \quad \quad a_{j}^t \geq x_{ij}^t \times (a_{i}^t + d_i + t_{ij})
$$

$$
\forall t \in T \quad \forall i \in C \quad \quad dt^0_i \leq a_{i}^t \leq dt^1_i \quad
$$


<aside style='background-color:rgba(245, 39, 39, 0.15)'><br>
    ‎ ‎ ‎  ⛔ The problem here is that we cannot multiply two decision variable together or we are no longer in a linear programmation problem.<div style='text-align:center' >$$\require{cancel}$$ $\cancel{{ij}^t \times (a_i^t + d_i + t_{ij})}$</div>
<br>
</aside>


**Linearization Technique**

Let introduce an auxiliary variable $z_{ij}^t$ to represent the product:

$z_{ij}^t = x_{ij}^t \cdot (a_{i}^t + d_i + t_{ij})$

Constraints of $a_j^t$:

- $z_{ij}^t \leq M \cdot x_{ij}^t \longrightarrow$ Ensures $z_{ij}^t = 0$ when $x_{ij}^t = 0$.
- $z_{ij}^t \leq a_{i}^t + d_i + t_{ij} \longrightarrow$ Ensures $z_{ij}^t$ does not exceed $a_{i}^t + d_i + t_{ij}$ when $x_{ij}^t = 1$.
- $z_{ij}^t \geq (a_{i}^t + d_i + t_{ij}) - M \cdot (1 - x_{ij}^t) \longrightarrow$ Ensures $z_{ij}^t = a_{i}^t + d_i + t_{ij}$ when $x_{ij}^t = 1$.
- $z_{ij}^t \geq 0 \longrightarrow$  Keeps $z_{ij}^t$ non-negative.

Here, $M$ is a sufficiently large constant (a big-$M$ parameter) that bounds the value of $a_{i}^t + d_i + t_{ij}$.

We can reformulate our time window constraint into:

$$
\forall i, j\in C^2 \quad i\neq j \quad \forall t \in T \quad \quad a_{j}^t \geq (a_{i}^t + d_i + t_{ij}) - M \cdot (1-x_{ij}^t)
$$

$$
\boxed{\forall t \in T \quad \forall i,j \in C \quad \quad dt^0_j \leq (a_{i}^t + d_i + t_{ij}) - M \cdot (1-x_{ij}^t) \leq dt^1_j}
$$

## 2.4 - Objective Function

---

(time distance to minimize)

$$
\sum_{t \in T} \sum_{i \in C} \sum_{j\in C} t_{ij} * x_{ij}^t * d_i
$$

 # 4. Solving Method

# 4.1 - Ant Colony Principle

---

The Ant Colony Optimization (ACO) method is a metaheuristic inspired by the foraging behavior of ants, where they find shortest paths between their colony and food sources using pheromones. In the context of the Vehicle Routing Problem (VRP), ACO is adapted to find optimal or near-optimal solutions for routing vehicles to serve a set of customers while minimizing costs such as distance, time, or fuel.

First, we initialize the ant colony. Each ant will explore paths that visit each client exactly once, always returning to the depot before its closing time, and then continuing with the remaining deliveries. For our problem, each part of one path starting and ending at the depot will represent one truck. The ants will leave pheromones on taken routes, so the more a route is chosen, the more pheromones it will have. The choice of the next route to take for an ant is random, with a greater probability to take a route with a shortest distance or with more pheromones. After all ants finish a path, best solutions will be rewarded with more pheromones while others' pheromones will evaporate a little. In our case, we are still going to reset the pheromones at some point so we do not get stuck on a path. Ants will keep improving their paths by exploring them through iterations, giving us a final solution converging to an optimal one.

---

**4.1.1 - Pheromone Trails**

- Each edge in the graph has a **pheromone level**, which reflects its desirability.
- Ants prefer paths with higher pheromone levels but also explore less-traveled paths to balance exploitation and exploration, by using a influenced random choice.

---

**4.1.2 - Heuristic Information**

Heuristic information complements pheromones and is typically based on the problem’s cost metric, e.g., the inverse of distance or travel time. This guides ants towards promising solutions.

---

**4.1.3 - Ant Solution Modeling**

Each ant builds a solution by:

1. **Starting at the depot.**
2. **Selecting the next customer** based on a probabilistic rule:

    $$
    P_{ij} = \frac{(\tau_{ij})^\alpha \cdot (\eta_{ij})^\beta}{\sum_{k \in \text{allowed}} (\tau_{ik})^\alpha \cdot (\eta_{ik})^\beta}
    $$

    - $\tau_{ij}$: Pheromone level on edge $ij$.
    - $\eta_{ij}$: Heuristic desirability of edge $ij$.
    - $\alpha$: Importance of pheromone.
    - $\beta$: Importance of heuristic.
    - **Allowed**: The set of feasible next nodes (based on constraints like capacity).
3. Updating the current vehicle’s route until its constraints are met, then starting a new route for the next vehicle.
4. Returning to the depot when all customers are visited.

---

**4.1.4 - Pheromone Update**

- **Pheromone evaporation**: Reduces pheromone levels to avoid stagnation and encourage exploration:

    $$
    \tau_{ij} = (1 - \rho) \cdot \tau_{ij}
    $$

    where  $\rho$ is the evaporation rate.

- **Pheromone deposition**: Ants that find better solutions deposit more pheromone:

    $$
    \tau_{ij} += \Delta \tau_{ij}
    $$

    $$
    \Delta \tau_{ij} = \begin{cases}Q / L & \text{if edge } ij \text{ is in the solution,} \\0 & \text{otherwise.}\end{cases}
    $$

    - $Q$: A constant.
    - $L$: Length (cost) of the ant’s solution.

---

**4.1.5 - Iterative Optimization**

The process is repeated for several iterations:

- In each iteration, multiple ants construct solutions.
- Pheromone trails are updated based on the quality of the solutions.
- Over time, good solutions receive higher pheromone reinforcement, guiding ants toward optimal or near-optimal solutions.

---

---

# 4.2 - **Advantages of ACO for VRP**

---

1. **Flexibility**: Easily incorporates various constraints of VRP (e.g., capacity, time windows).
2. **Scalability**: Works well for large, complex instances.
3. **Adaptiveness**: Can balance between exploration (finding new solutions) and exploitation (improving known solutions).

---

# 4.3 - Advantages for VRPTW

---

Ant Colony Optimization (ACO) is particularly relevant for solving the Vehicle Routing Problem with Time Windows (VRPTW) due to its nature and flexibility:

1. **Adaptability to Constraints**: ACO can incorporate complex constraints like time windows, vehicle capacities, and distance limits. The pheromone update rules and heuristic functions allow fine-tuning to respect these constraints.
2. **Distributed Decision-Making**: In VRPTW, multiple vehicles must make routing decisions. ACO mimics this by having multiple “ants” build routes simultaneously, which aligns well with the problem’s distributed nature.
3. **Optimization through Collaboration**: The pheromone trails encourage solutions that are both locally and globally efficient. This helps balance between minimizing travel distance and respecting time windows.
4. **Scalability**: ACO performs well with increasing problem sizes and complexities, making it suitable for large-scale VRPTW instances.
5. **Dynamic Problem Solving**: VRPTW often involves dynamic elements, such as changes in time windows or demands. ACO’s iterative nature allows it to adapt to these changes during the optimization process.

These features make ACO an effective and popular method for solving VRPTW problems in logistics and transportation.

---

# **4.4 - Challenges**

---

1. **Parameter tuning** $( \alpha ,  \beta ,  \rho ,  Q )$d significantly affects performance.
2. **Computational cost**: Can be high for large instances due to repeated solution construction and pheromone updates.

In [4]:
import pandas as pd
import itertools
import random

from src.objects.customer import Customer
from src.aco.aco import ACO
from src.nearest_neighbor import NNO
import src.printer as pr

In [5]:
def instance_generator(customer_count, rand_seed=0):
    """
    Generates a random instance of the vehicle routing problem.

    Returns:
    tuple: List of customer objects and the depot object., number of trucks, capacity of each truck
    """
    random.seed(rand_seed)  # Set the random seed for reproducibility
    
    customers = []
    
    # Number of trucks
    truck_count = random.randint(10, 75)
    
    # Capacity of each truck
    capacity = random.randint(10, 50) * 10

    # Number of customers
    num_customers = customer_count
    
    # Generate random service time for the customer
    service_time = random.randint(1, 5) * 10

    for i in range(1, num_customers + 1):

        # Generate random demand for the customer
        demand = random.randint(1, 10) * 10
        
        # Generate random coordinates for the customer
        x_coord = random.uniform(0, 100)
        y_coord = random.uniform(0, 100)

        # Generate random ready time for the customer
        ready_time = random.randint(0, 1000)

        # Generate random due date for the customer
        due_date = ready_time + random.randint(100, 300)

        # Create a Customer object and add it to the list of customers
        customers.append(Customer(i, x_coord, y_coord, demand, ready_time, due_date, service_time))

    # Generate the depot (warehouse) at the center of the customers
    depot = Customer(0, sum(customer.x_coord for customer in customers) / num_customers, sum(customer.y_coord for customer in customers) / num_customers, 0, 0, max(x.due_date for x in customers) + service_time + 100, 0)

    return [depot] + customers, truck_count, capacity

In [6]:
def read_dataset(file_path):
    print(f"Reading dataset -> {file_path}")
    # Reading the vehicle data (lines 3-4)
    vehicle_df = pd.read_csv(file_path, skiprows=4, nrows=1, sep='\\s+', names=['Number', 'Capacity'])
    
    # Reading the customer data (after line 9)
    customers_df = pd.read_csv(file_path, skiprows=9, sep='\\s+', names=['Cust No.', 'XCoord.', 'YCoord.', 'Demand', 'Ready Time', 'Due Date', 'Service Time'])

    truck_count = vehicle_df['Number'][0]  # Get the number of trucks
    capacity = vehicle_df['Capacity'][0]  # Get the capacity of each truck

    # Create a list of Customer objects from the customer data
    customers = [Customer(row['Cust No.'], row['XCoord.'], row['YCoord.'], row['Demand'], row['Ready Time'], row['Due Date'], row['Service Time']) for index, row in customers_df.iterrows()]
    
    return customers, truck_count, capacity

In [7]:
def main_aco_run(file_path, num_ants, num_iterations, pheromon_importance_alpha, heuristic_importance_beta, evaporation_rho, pheromon_init=-1):
    customers, truck_count, capacity = read_dataset(file_path)  # Read the dataset from the input file
    
    if pheromon_init == -1:
        nno = NNO(customers)
        distance = nno.run()
        pheromon_init = 1/distance

    # Print the vehicle data
    print(f"Number of trucks: {truck_count}")
    print(f"Capacity of each truck: {capacity}")
    print(f"Number of customers: {len(customers) - 1}")

    # Initialize the Ant Colony Optimization (ACO) algorithm
    aco = ACO(
        customers.copy(),
        truck_count=truck_count,
        truck_capacity=capacity,
        iterations=num_iterations,
        ants_count=num_ants,
        pheromone_importance=pheromon_importance_alpha,
        heuristic_importance=heuristic_importance_beta,
        evaporation_rate=evaporation_rho,
        pheromone_init=pheromon_init,
        debug=False
    )

    # Run the ACO algorithm to find the best solution
    aco.run()
    
    return aco.get_results()

In [8]:
def test_parameters_to_excel(file_path, excel_name, num_ants_list, num_iterations_list, pheromon_importance_alpha_list, heuristic_importance_beta_list, evaporation_rho_list, pheromon_init_list):
    customers, truck_count, capacity = read_dataset(file_path=file_path) # Read the dataset from the input file
        
    # Print the vehicle data
    print(f"Number of trucks: {truck_count}")
    print(f"Capacity of each truck: {capacity}")
    print(f"Number of customers: {len(customers) - 1}")
    
    param_combinations = itertools.product(
        num_ants_list, num_iterations_list, pheromon_importance_alpha_list, heuristic_importance_beta_list, evaporation_rho_list, pheromon_init_list
    )
    
    (index_column,
     num_ants_column,
     num_iterations_column,
     pheromon_importance_alpha_column,
     heuristic_importance_beta_column,
     evaporation_rho_column,
     pheromon_init_column,
     truck_count_column,
     lowest_distance_column,
     average_distance_column,
     best_distance_column
     ) = [], [], [], [], [], [], [], [], [], [], []
    
    i = 1
    product_len = len(num_ants_list) * len(num_iterations_list) * len(pheromon_importance_alpha_list) * len(heuristic_importance_beta_list) * len(evaporation_rho_list) * len(pheromon_init_list)
    
    for num_ants, num_iterations, pheromon_importance_alpha, heuristic_importance_beta, evaporation_rho, pheromon_init in param_combinations:
        print(f"Iteration N°{i}/{product_len}")
        print("-" * 100)
        print(f"Ant count: {num_ants}")
        print(f"Iteration count: {num_iterations}")
        print(f"Pheromon importance (alpha): {pheromon_importance_alpha}")
        print(f"Heuristic importance (beta): {heuristic_importance_beta}")
        print(f"Evaporation rate (rho): {evaporation_rho}")
        print(f"Initial pheromone level: {pheromon_init}")        
        print("-" * 100)

        # Run the ACO algorithm with the current parameters
        (customers,
         depot, costs, best_solution, used_truck_count, best_distance, total_time) = main_aco_run(
            file_path='dataset/c101.txt',
            num_ants=num_ants,
            num_iterations=num_iterations,
            pheromon_importance_alpha=pheromon_importance_alpha,
            heuristic_importance_beta=heuristic_importance_beta,
            evaporation_rho=evaporation_rho,
            pheromon_init=pheromon_init)
        
        index_column.append(i)
        num_ants_column.append(num_ants)
        num_iterations_column.append(num_iterations)
        pheromon_importance_alpha_column.append(pheromon_importance_alpha)
        heuristic_importance_beta_column.append(heuristic_importance_beta)
        evaporation_rho_column.append(evaporation_rho)
        pheromon_init_column.append(pheromon_init)
        truck_count_column.append(len(best_solution))
        lowest_distance_column.append(min(costs))
        average_distance_column.append(sum(costs) / len(costs))
        best_distance_column.append(max(costs))

        # Print the truck usage details
        pr.print_truck_usage(best_solution, depot, show_graphics=False)
        
        i += 1
    
    excel_df = pd.DataFrame({"Ant count": num_ants_column,
                             "Iteration count": num_iterations_column,
                             "Pheromon importance (alpha)": pheromon_importance_alpha_column,
                             "Heuristic importance (beta)": heuristic_importance_beta_column,
                             "Evaporation rate (rho)": evaporation_rho_column,
                             "Initial pheromone level": pheromon_init_column,
                             "Lowest distance": lowest_distance_column,
                             "Average distance": average_distance_column,
                             "Best distance": best_distance_column})
    
    excel_df.to_excel(excel_name, index=True)  # Save the results to an Excel file
  
def main():
    (customers, depot, costs, best_solution, used_truck_count, total_distance, total_time) = main_aco_run(
        file_path='dataset/c103.txt',
        num_ants=50,
        num_iterations=50,
        pheromon_importance_alpha=1.0,
        heuristic_importance_beta=2.0,
        evaporation_rho=0.1,
        pheromon_init=-1)
    
    # Print the solution details
    pr.print_aco_solution(used_truck_count, total_distance, total_time)

    # Plot the routes of the best solution
    pr.plot_routes(best_solution, depot)

    # Print the cost history
    pr.print_costs_history(costs)

    # Print the truck usage details
    pr.print_truck_usage(best_solution, depot, show_graphics=False)

def multiple_runs_analysis(data_file_path, excel_output_name, num_runs=50, show_graphics=False):
    distances_column = []

    customers, truck_count, capacity = read_dataset(data_file_path)  # Read the dataset from the input file

    # Print the vehicle data
    print(f"Number of trucks: {truck_count}")
    print(f"Capacity of each truck: {capacity}")
    print(f"Number of customers: {len(customers) - 1}")

    for i in range(num_runs):
        print(f"Run N°{i + 1}/{num_runs}")
        # Initialize the Ant Colony Optimization (ACO) algorithm
        (customers, depot, costs, best_solution, used_truck_count, best_distance, total_time) = main_aco_run(
        file_path='dataset/c103.txt',
        num_ants=20,
        num_iterations=20,
        pheromon_importance_alpha=1.0,
        heuristic_importance_beta=2.0,
        evaporation_rho=0.1,
        pheromon_init=-1)

        distances_column.append(best_distance)

        if show_graphics:
            # Plot the routes of the best solution
            pr.plot_routes(best_solution, depot)
        
        # Print the solution details
        pr.print_aco_solution(used_truck_count, best_distance, total_time)

    writer = pd.ExcelWriter(excel_output_name, engine='xlsxwriter')
    excel_df = pd.DataFrame({"Distances": distances_column})
    excel_df.to_excel(writer, index=True)  # Save the results to an Excel file
    
    print(f"Results saved to {excel_output_name}")

if __name__ == '__main__':
    # test_parameters_to_excel(
    #     file_path='dataset/r111.txt',
    #     excel_name='paramteres_result.xlsx',
    #     num_ants_list=[20, 50, 100],
    #     num_iterations_list=[20, 50, 100],
    #     pheromon_importance_alpha_list=[1.0, 2.0, 3.0],
    #     heuristic_importance_beta_list=[1.0, 2.0, 3.0],
    #     evaporation_rho_list=[0.1, 0.3, 0.5],
    #     pheromon_init_list=[-1]
    # )
    
    # main()
    
    multiple_runs_analysis('dataset/r111.txt', 'results_analysis.xlsx', num_runs=50)

Reading dataset -> dataset/r111.txt
Number of trucks: 25
Capacity of each truck: 200
Number of customers: 100
Run N°1/50
Reading dataset -> dataset/c103.txt
--------------------------------------------------
Nearest Neighbor Solution:
Total distance traveled by the trucks: 51.48
Path: depot -> 20 -> 21 -> 22 -> 23 -> 26 -> 28 -> 27 -> 25 -> 24 -> 29 -> 30 -> 34 -> 36 -> 39 -> 38 -> 37 -> 35 -> 31 -> 32 -> 33 -> 52 -> 49 -> 47 -> 43 -> 42 -> 41 -> 40 -> 44 -> 45 -> 46 -> 48 -> 50 -> 51 -> 59 -> 57 -> 55 -> 54 -> 53 -> 56 -> 58 -> 60 -> 68 -> 64 -> 61 -> 72 -> 74 -> 62 -> 66 -> 69 -> 67 -> 65 -> 63 -> 90 -> 89 -> 88 -> 85 -> 84 -> 83 -> 82 -> 86 -> 87 -> 91 -> 98 -> 96 -> 95 -> 94 -> 93 -> 92 -> 97 -> 100 -> 99 -> 2 -> 1 -> 4 -> 3 -> 5 -> 7 -> 8 -> 9 -> 6 -> 11 -> 10 -> 75 -> 13 -> 17 -> 18 -> 19 -> 15 -> 16 -> 14 -> 12 -> 81 -> 78 -> 76 -> 71 -> 70 -> 73 -> 77 -> 79 -> depot
--------------------------------------------------
Number of trucks: 25
Capacity of each truck: 200
Number of cus

AttributeError: 'ACO' object has no attribute 'get_results'

# Ressources

<br>

VRP :

- [https://en.wikipedia.org/wiki/Vehicle_routing_problem](https://en.wikipedia.org/wiki/Vehicle_routing_problem)

- [https://www.sciencedirect.com/science/article/abs/pii/S0360835215004775](https://www.sciencedirect.com/science/article/abs/pii/S0360835215004775)

- [https://www.youtube.com/watch?v=v5kaDxG5w30](https://www.youtube.com/watch?v=v5kaDxG5w30)

ACO : [https://ieeexplore.ieee.org/abstract/document/5453634](https://ieeexplore.ieee.org/abstract/document/5453634)

Linear programming : [https://www.youtube.com/watch?v=E72DWgKP_1Y&list=LL&index=7&t=66s](https://www.youtube.com/watch?v=E72DWgKP_1Y&list=LL&index=7&t=66s)

Complexity classes (NP, NP-Hard, NP-Complete, etc.) :

- [https://www.geeksforgeeks.org/difference-between-np-hard-and-np-complete-problem/](https://www.geeksforgeeks.org/difference-between-np-hard-and-np-complete-problem/)

- [https://introtcs.org/public/lec_12_NP.html](https://introtcs.org/public/lec_12_NP.html)

VRP Solomon Datasets Benchmarks : [http://vrp.galgos.inf.puc-rio.br/index.php/en/](http://vrp.galgos.inf.puc-rio.br/index.php/en/)